In [13]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html

# ----------------------------------------------------------------
# 1) Load Data from CSV File
# ----------------------------------------------------------------
print("Loading data from CSV file...")
try:
    # Load the combined dataset
    df_combined = pd.read_csv('DO_measurements_and_predictions_V0.csv')
    print("Data loaded successfully.")
except Exception as e:
    print(f"Error loading data: {e}")
    print("Make sure to run data_processor.py first to generate the required CSV file.")
    import sys
    sys.exit(1)

# Import required libraries for linear regression
import numpy as np
from sklearn.linear_model import LinearRegression

# Convert Date column to datetime
df_combined['Date'] = pd.to_datetime(df_combined['Date'])

# Hard-code DO ranges
morn_lower, morn_upper = 3, 5  # Morning acceptable DO range
even_lower, even_upper = 8, 12  # Evening acceptable DO range

# Split data into morning and evening datasets
df_morning = df_combined[df_combined['Measurement Type'] == 'Morning'].copy()
df_evening = df_combined[df_combined['Measurement Type'] == 'Evening'].copy()

# Calculate date range for plotting
date_min = df_combined['Date'].min()
date_max = df_combined['Date'].max()

# Get list of pond IDs
pond_ids = sorted(df_combined['Pond ID'].unique())
initial_pond = pond_ids[0]

# Calculate R² values for each pond
from sklearn.metrics import r2_score
pond_r2_values = {}

for pond in pond_ids:
    # Morning R²
    df_morn_pond = df_morning[df_morning['Pond ID'] == pond]
    if len(df_morn_pond) > 1:
        r2_morn = r2_score(df_morn_pond['DO Actual'], df_morn_pond['DO Predicted'])
    else:
        r2_morn = float('nan')

    # Evening R²
    df_even_pond = df_evening[df_evening['Pond ID'] == pond]
    if len(df_even_pond) > 1:
        r2_even = r2_score(df_even_pond['DO Actual'], df_even_pond['DO Predicted'])
    else:
        r2_even = float('nan')

    pond_r2_values[pond] = (r2_morn, r2_even)

print(f"Found {len(pond_ids)} ponds. Initial pond: {initial_pond}")

# ----------------------------------------------------------------
# 2) Create Subplots (6 rows)
# ----------------------------------------------------------------
print("Creating visualization...")
fig = make_subplots(
    rows=6, cols=1,
    subplot_titles=[
        "Morning DO",
        "Morning DO Exceedance",
        "Evening DO",
        "Evening DO Exceedance",
        "Morning (Pred vs. Actual)",
        "Evening (Pred vs. Actual)"
    ],
    vertical_spacing=0.05,
)

# Filter data for initial pond
df_morn_init = df_morning[df_morning['Pond ID'] == initial_pond].copy()
df_even_init = df_evening[df_evening['Pond ID'] == initial_pond].copy()

# Constants for plotting
DAY_MS = 24 * 60 * 60 * 1000
HALF_DAY_MS = DAY_MS / 2

# Calculate exceedances for each measurement
def calc_exceedance(value, lower, upper):
    if value < lower:
        return value - lower
    elif value > upper:
        return value - upper
    else:
        return 0

# Calculate exceedances for morning data
df_morning['Actual_Exceed'] = df_morning['DO Actual'].apply(lambda v: calc_exceedance(v, morn_lower, morn_upper))
df_morning['Predicted_Exceed'] = df_morning['DO Predicted'].apply(lambda v: calc_exceedance(v, morn_lower, morn_upper))

# Calculate exceedances for evening data
df_evening['Actual_Exceed'] = df_evening['DO Actual'].apply(lambda v: calc_exceedance(v, even_lower, even_upper))
df_evening['Predicted_Exceed'] = df_evening['DO Predicted'].apply(lambda v: calc_exceedance(v, even_lower, even_upper))

# Update init datasets with exceedance data
df_morn_init = df_morning[df_morning['Pond ID'] == initial_pond].copy()
df_even_init = df_evening[df_evening['Pond ID'] == initial_pond].copy()

# ----------------------------------------------------------------
# Helper function for calculating trendlines
# ----------------------------------------------------------------
def calc_trendline(x_data, y_data, x_min=0, x_max=None):
    """Calculate linear regression and return parameters

    Args:
        x_data: Array of x values
        y_data: Array of y values
        x_min: Minimum x value for trendline (default: 0)
        x_max: Maximum x value for trendline (defaults to axis max)
    """
    if len(x_data) < 2:
        return None, None, None, []

    X = np.array(x_data).reshape(-1, 1)
    y = np.array(y_data)

    try:
        model = LinearRegression()
        model.fit(X, y)
        slope = model.coef_[0]
        intercept = model.intercept_

        if x_max is None:
            x_max = 8.0 if max(x_data) < 8 else 20.0

        x_range = [x_min, x_max]
        y_range = [slope * x + intercept for x in x_range]
        equation = f"y = {slope:.2f}x + {intercept:.2f}"
        return slope, intercept, equation, list(zip(x_range, y_range))
    except Exception as e:
        print(f"Error calculating trendline: {e}")
        return None, None, None, []

# ----------------------------------------------------------------
# 3) Morning DO (row=1)
# ----------------------------------------------------------------
trace_morn_actual = go.Scatter(
    x=df_morn_init['Date'],
    y=df_morn_init['DO Actual'],
    mode='markers',
    name='Actual DO',
    marker=dict(color='blue')
)
trace_morn_pred = go.Scatter(
    x=df_morn_init['Date'],
    y=df_morn_init['DO Predicted'],
    mode='markers',
    name='Predicted DO',
    marker=dict(color='red')
)

fig.add_trace(trace_morn_actual, row=1, col=1)
fig.add_trace(trace_morn_pred, row=1, col=1)

fig.add_shape(
    type="rect",
    xref="x", yref="y",
    x0=date_min, x1=date_max,
    y0=morn_lower, y1=morn_upper,
    fillcolor="rgba(0, 100, 0, 0.2)",
    line_width=0,
    layer="below",
    row=1, col=1
)
fig.update_yaxes(title="Morning DO (mg/L)", range=[0, 8], row=1, col=1)

# ----------------------------------------------------------------
# 4) Morning DO Exceedance (row=2)
# ----------------------------------------------------------------
trace_morn_ex_actual = go.Bar(
    x=df_morn_init['Date'],
    y=df_morn_init['Actual_Exceed'],
    name='Actual Exceedance',
    marker_color='blue',
    offsetgroup='morning_actual',
    width=HALF_DAY_MS
)
trace_morn_ex_pred = go.Bar(
    x=df_morn_init['Date'],
    y=df_morn_init['Predicted_Exceed'],
    name='Predicted Exceedance',
    marker_color='red',
    offsetgroup='morning_predicted',
    width=HALF_DAY_MS
)

fig.add_trace(trace_morn_ex_actual, row=2, col=1)
fig.add_trace(trace_morn_ex_pred, row=2, col=1)
fig.update_yaxes(title="Morning DO Exceedance (mg/L)", range=[-4, 4], row=2, col=1)

# ----------------------------------------------------------------
# 5) Evening DO (row=3)
# ----------------------------------------------------------------
trace_even_actual = go.Scatter(
    x=df_even_init['Date'],
    y=df_even_init['DO Actual'],
    mode='markers',
    name='Actual DO',
    marker=dict(color='blue'),
    showlegend=False
)
trace_even_pred = go.Scatter(
    x=df_even_init['Date'],
    y=df_even_init['DO Predicted'],
    mode='markers',
    name='Predicted DO',
    marker=dict(color='red'),
    showlegend=False
)

fig.add_trace(trace_even_actual, row=3, col=1)
fig.add_trace(trace_even_pred, row=3, col=1)

fig.add_shape(
    type="rect",
    xref="x3", yref="y3",
    x0=date_min, x1=date_max,
    y0=even_lower, y1=even_upper,
    fillcolor="rgba(0, 100, 0, 0.2)",
    line_width=0,
    layer="below",
    row=3, col=1
)
fig.update_yaxes(title="Evening DO (mg/L)", range=[0, 20], row=3, col=1)

# ----------------------------------------------------------------
# 6) Evening DO Exceedance (row=4)
# ----------------------------------------------------------------
trace_even_ex_actual = go.Bar(
    x=df_even_init['Date'],
    y=df_even_init['Actual_Exceed'],
    name='Evening Actual',
    marker_color='blue',
    offsetgroup='evening_actual',
    width=HALF_DAY_MS,
    showlegend=False
)
trace_even_ex_pred = go.Bar(
    x=df_even_init['Date'],
    y=df_even_init['Predicted_Exceed'],
    name='Evening Predicted',
    marker_color='red',
    offsetgroup='evening_predicted',
    width=HALF_DAY_MS,
    showlegend=False
)

fig.add_trace(trace_even_ex_actual, row=4, col=1)
fig.add_trace(trace_even_ex_pred, row=4, col=1)
fig.update_yaxes(title="Evening DO Exceedance (mg/L)", range=[-10, 10], row=4, col=1)

# ----------------------------------------------------------------
# 7) Morning (Pred vs. Actual) (row=5)
# ----------------------------------------------------------------
slope_morn, intercept_morn, equation_morn, trendline_morn = calc_trendline(
    df_morn_init['DO Actual'], df_morn_init['DO Predicted'], x_min=0, x_max=8
)

trace_morn_scatter_init = go.Scatter(
    x=df_morn_init['DO Actual'],
    y=df_morn_init['DO Predicted'],
    mode='markers',
    name='Morning (Act→Pred)',
    marker=dict(color='green'),
    showlegend=True
)
fig.add_trace(trace_morn_scatter_init, row=5, col=1)

if trendline_morn:
    x_trend_m, y_trend_m = zip(*trendline_morn)
    trace_morn_trendline = go.Scatter(
        x=x_trend_m,
        y=y_trend_m,
        mode='lines',
        name='Morning Trendline',
        line=dict(color='darkgreen', dash='dash', width=3),
        showlegend=False
    )
    fig.add_trace(trace_morn_trendline, row=5, col=1)

fig.add_shape(
    type="line",
    xref="x5", yref="y5",
    x0=0, y0=0, x1=8, y1=8,
    line=dict(color="black", dash="dash"),
    row=5, col=1
)

if equation_morn:
    fig.add_annotation(
        text=equation_morn,
        x=0.05,
        y=0.95,
        xref="x5 domain",
        yref="y5 domain",
        showarrow=False,
        font=dict(size=12, color="darkgreen"),
        bgcolor="rgba(255, 255, 255, 0.7)",
        bordercolor="darkgreen",
        borderwidth=1,
        borderpad=4,
        row=5, col=1
    )

# ----------------------------------------------------------------
# 8) Evening (Pred vs. Actual) (row=6)
# ----------------------------------------------------------------
slope_even, intercept_even, equation_even, trendline_even = calc_trendline(
    df_even_init['DO Actual'], df_even_init['DO Predicted'], x_min=0, x_max=20
)

trace_even_scatter_init = go.Scatter(
    x=df_even_init['DO Actual'],
    y=df_even_init['DO Predicted'],
    mode='markers',
    name='Evening (Act→Pred)',
    marker=dict(color='orange'),
    showlegend=True
)

if trendline_even:
    x_trend, y_trend = zip(*trendline_even)
    trace_even_trendline = go.Scatter(
        x=x_trend,
        y=y_trend,
        mode='lines',
        name='Evening Trendline',
        line=dict(color='darkorange', dash='dash', width=3),
        showlegend=False
    )
    fig.add_trace(trace_even_trendline, row=6, col=1)

fig.add_trace(trace_even_scatter_init, row=6, col=1)

fig.add_shape(
    type="line",
    xref="x6", yref="y6",
    x0=0, y0=0, x1=20, y1=20,
    line=dict(color="black", dash="dash", width=1),
    row=6, col=1
)

if equation_even:
    fig.add_annotation(
        text=equation_even,
        x=0.05,
        y=0.95,
        xref="x6 domain",
        yref="y6 domain",
        showarrow=False,
        font=dict(size=12, color="darkorange"),
        bgcolor="rgba(255, 255, 255, 0.7)",
        bordercolor="darkorange",
        borderwidth=1,
        borderpad=4,
        row=6, col=1
    )

# ----------------------------------------------------------------
# 9) Add Initial Annotations for the Starting Pond
# ----------------------------------------------------------------
r2_morn_init, r2_even_init = pond_r2_values[initial_pond]

fig.add_annotation(
    text=f"Morning R² = {r2_morn_init:.3f}",
    x=0.55,
    y=1.00,
    xref="x5 domain",
    yref="y5 domain",
    xanchor="right",
    yanchor="top",
    showarrow=False,
    row=5, col=1
)

fig.add_annotation(
    text=f"Evening R² = {r2_even_init:.3f}",
    x=0.55,
    y=1.00,
    xref="x6 domain",
    yref="y6 domain",
    xanchor="right",
    yanchor="top",
    showarrow=False,
    row=6, col=1
)

# ----------------------------------------------------------------
# 10) Build Dropdown Buttons for Pond Selection
# ----------------------------------------------------------------
print(f"Creating dropdown menu with {len(pond_ids)} ponds...")
dropdown_buttons = []
for pond in pond_ids:
    df_morn = df_morning[df_morning['Pond ID'] == pond]
    df_even = df_evening[df_evening['Pond ID'] == pond]
    r2_morn, r2_even = pond_r2_values[pond]

    slope_morn_pond, intercept_morn_pond, equation_morn_pond, trendline_morn_pond = calc_trendline(
        df_morn['DO Actual'], df_morn['DO Predicted'], x_min=0, x_max=8
    )

    slope_even_pond, intercept_even_pond, equation_even_pond, trendline_even_pond = calc_trendline(
        df_even['DO Actual'], df_even['DO Predicted'], x_min=0, x_max=20
    )

    if trendline_morn_pond:
        x_trend_morn, y_trend_morn = zip(*trendline_morn_pond)
    else:
        x_trend_morn, y_trend_morn = [0, 8], [0, 0]

    if trendline_even_pond:
        x_trend_even, y_trend_even = zip(*trendline_even_pond)
    else:
        x_trend_even, y_trend_even = [0, 20], [0, 0]

    annotations = []
    annotations.append(dict(
        text=f"Morning R² = {r2_morn:.3f}",
        x=0.55,
        y=1.00,
        xref="x5 domain",
        yref="y5 domain",
        xanchor="right",
        yanchor="top",
        showarrow=False,
    ))
    annotations.append(dict(
        text=f"Evening R² = {r2_even:.3f}",
        x=0.55,
        y=1.00,
        xref="x6 domain",
        yref="y6 domain",
        xanchor="right",
        yanchor="top",
        showarrow=False,
    ))
    if equation_morn_pond:
        annotations.append(dict(
            text=equation_morn_pond,
            x=0.05,
            y=0.95,
            xref="x5 domain",
            yref="y5 domain",
            showarrow=False,
            font=dict(size=12, color="darkgreen"),
            bgcolor="rgba(255, 255, 255, 0.7)",
            bordercolor="darkgreen",
            borderwidth=1,
            borderpad=4
        ))
    if equation_even_pond:
        annotations.append(dict(
            text=equation_even_pond,
            x=0.05,
            y=0.95,
            xref="x6 domain",
            yref="y6 domain",
            showarrow=False,
            font=dict(size=12, color="darkorange"),
            bgcolor="rgba(255, 255, 255, 0.7)",
            bordercolor="darkorange",
            borderwidth=1,
            borderpad=4
        ))

    button = dict(
        label=str(pond),
        method="update",
        args=[
            {
                "x": [
                    df_morn['Date'], df_morn['Date'],              # row=1
                    df_morn['Date'], df_morn['Date'],              # row=2
                    df_even['Date'], df_even['Date'],              # row=3
                    df_even['Date'], df_even['Date'],              # row=4
                    df_morn['DO Actual'], x_trend_morn,            # row=5
                    x_trend_even, df_even['DO Actual']             # row=6 (trendline, scatter)
                ],
                "y": [
                    df_morn['DO Actual'], df_morn['DO Predicted'],         # row=1
                    df_morn['Actual_Exceed'], df_morn['Predicted_Exceed'], # row=2
                    df_even['DO Actual'], df_even['DO Predicted'],         # row=3
                    df_even['Actual_Exceed'], df_even['Predicted_Exceed'], # row=4
                    df_morn['DO Predicted'], y_trend_morn,                 # row=5
                    y_trend_even, df_even['DO Predicted']                  # row=6 (trendline, scatter)
                ]
            },
            {
                "annotations": annotations,
                "xaxis5.range": [0, 8],
                "xaxis5.autorange": False,
                "xaxis6.range": [0, 20],
                "xaxis6.autorange": False
            }
        ]
    )
    dropdown_buttons.append(button)

# ----------------------------------------------------------------
# 11) Final Layout Setup
# ----------------------------------------------------------------
fig.update_layout(
    updatemenus=[{
        "buttons": dropdown_buttons,
        "direction": "down",
        "showactive": True,
        "x": 1.01,
        "xanchor": "left",
        "y": 1.00,
        "yanchor": "top"
    }],
    xaxis=dict(matches="x4", showticklabels=True),
    xaxis2=dict(matches="x4", showticklabels=True),
    xaxis3=dict(matches="x4", showticklabels=True),
    xaxis4=dict(
        rangeslider=dict(visible=False, thickness=0.1),
        type="date"
    ),
    xaxis5=dict(
        title="Actual DO (mg/L)",
        range=[0, 8],
        autorange=False,
        showticklabels=True,
        title_standoff=0,
        automargin=True
    ),
    xaxis6=dict(
        title="Actual DO (mg/L)",
        range=[0, 20],
        autorange=False,
        showticklabels=True,
        title_standoff=0,
        automargin=True
    ),
    barmode='group',
    bargap=0.2,
    bargroupgap=0.0,
    legend=dict(
        x=1.0,
        y=0.98,
        xanchor="left",
        yanchor="top",
        orientation="v"
    ),
    yaxis5=dict(title="Predicted DO (mg/L)"),
    yaxis6=dict(title="Predicted DO (mg/L)"),
    height=1700,
    width=950,
    margin=dict(
      l=100,
      r=20,
      t=50,
      b=50
    )
)

# ----------------------------------------------------------------
# 12) Initialize the Dash App
# ----------------------------------------------------------------
print("Setting up Dash app...")
app = Dash(__name__)

app.layout = html.Div([
    html.H1("DO Predictions Dashboard"),
    html.P("Select a pond from the dropdown menu in the top-right corner of the chart."),
    dcc.Graph(figure=fig)
])

# ----------------------------------------------------------------
# 13) Run the App
# ----------------------------------------------------------------
if __name__ == '__main__':
    print("Starting Dash server. Access the dashboard at http://127.0.0.1:8055/")
    app.run_server(debug=True, port=8055)

Loading data from CSV file...
Data loaded successfully.
Found 3 ponds. Initial pond: WG-AJU1
Creating visualization...
Creating dropdown menu with 3 ponds...
Setting up Dash app...
Starting Dash server. Access the dashboard at http://127.0.0.1:8055/
